In [10]:
import os
import gmaps
import gmaps.datasets
import pandas as pd

In [7]:
%pylab inline

# See the https://jupyter-gmaps.readthedocs.io/en/latest/install.html for installation
# set GOOGLE_API_KEY in your environment
gmaps.configure(api_key=os.environ["GOOGLE_API_KEY"])

Populating the interactive namespace from numpy and matplotlib


### Loading Data

In [ ]:
ride_data = pd.read_csv

In [8]:
# load a Numpy array of (latitude, longitude) pairs
locations = gmaps.datasets.load_dataset("taxi_rides")

In [9]:
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations))
fig

Figure(layout=FigureLayout(height='420px'))